# Projeto Final Redes Neurais: Propriedades Mecânicas de Materiais

#### Trio: csv_Computeiros_sinápticos_virtuais 

#### Integrantes: Diogo Pereira de Lima Carvalho, José David Sales e Mayllon Emmanoel Pequeno

<p style="text-align: justify;"> O notebook em questão consiste no processo de otimização de hiperparâmetros da rede neural MLP. Para isso, utilizou-se o <code>optuna</code>, baseando-se em um notebook do professor Daniel Roberto Cassar [1]. Para essa otimização, considerou-se variar o número de camadas ocultas, o número de neurônios por camada oculta e se os neurôneos de cada camada apresentam ou não viés.

<p style="text-align: justify;"> Abaixo, foram importadas as bibliotecas necessárias e determinadas as constantes utilizadas neste notebook.

In [1]:
import lightning as L
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import stats
from sklearn.metrics import root_mean_squared_error
from objetos_otimizacao_rede_neural import *
from lightning.pytorch.loggers import TensorBoardLogger
from optuna import create_study, Trial
import os
from IPython.display import clear_output

In [4]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 0
NUM_TENTATIVAS_OTIMIZACAO = 500
NUM_EPOCAS = 30

<p style="text-align: justify;"> Abaixo, foi utilizado a instância <code>logger</code> para guardar informações importantes sobre as métricas de desempenho do modelo que será treinado. Um <code>treinador</code> também é criado que será responsável pelos ciclos de treinamamento do modelo.

In [5]:
logger = TensorBoardLogger(save_dir=os.getcwd(), version=1, name="lightning_logs/")
treinador = L.Trainer(logger=False,enable_checkpointing=True,max_epochs=NUM_EPOCAS)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


<p style="text-align: justify;"> Abaixo, a função <code>build_model</code> cria uma instância do modelo, em que há 5 dados de entrada (5 atributos) e 1 dado de saída, já que se possui apenas um <i>target</i> numérico. Nisso, em cada tentativa, pode-se variar o número de camadas (entre 2 e 5), o número de neurônios por camada (de 2 a 15) e se haverá ou não viés para neurônios de cada camada. 

In [6]:
def build_model(trial):
    num_dados_de_entrada = 5
    num_dados_de_saida = 1

    n_layers = trial.suggest_int("n_layers", 2, 5) 
    neuronios_camadas = []
    vieses = []

    for i in range(n_layers):
        neuronios =  trial.suggest_int(f"c{i}", 2, 15)
        bia = trial.suggest_categorical(f"bias{i}", [False, True])
        neuronios_camadas.append(neuronios)
        vieses.append(bia)
        
    bia = trial.suggest_categorical(f"bia out", [False, True])
    vieses.append(bia)
    
    minha_mlp = MLP(
        num_dados_de_entrada, list(neuronios_camadas), list(vieses), num_dados_de_saida
    )
    
    return minha_mlp

<p style="text-align: justify;"> Abaixo, é defido a função <code>objective</code>, responsável por computar a métrica RMSE para a MLP com uma validação cruzada realizada a partir da iteração abaixo.

In [8]:
def objective(trial):

    k = []
    for i in range(int(1/TAMANHO_TESTE)):
        print(i)
        dm = DataModule(TAMANHO_TESTE, SEMENTE_ALEATORIA, i)
        minha_mlp = build_model(trial)
        treinador.fit(minha_mlp, dm)

        minha_mlp.eval()
        dm.setup("test")

        with torch.no_grad():
            X_true = dm.X_teste

            y_true = dm.y_teste
            y_true = dm.y_scaler.inverse_transform(y_true)

            y_pred = minha_mlp(X_true)
            y_pred = dm.y_scaler.inverse_transform(y_pred)

            RMSE = root_mean_squared_error(y_true, y_pred)
        
        k.append(RMSE)

    rmse_medio = (sum(np.array(k)**2)/int(1/TAMANHO_TESTE))**.5
    return rmse_medio

<p style="text-align: justify;"> Por fim, criou-se a instância de estudo de <code>create_study</code>, sendo utilizado para resolver um problema de minimização. A partir disso, realia-se um certo número de tentaticas de modelos conforme o número determinado na constante <code>NUM_TENTATIVAS_OTIMIZACAO</code>.

In [6]:
#%%capture
study = create_study(direction='minimize')
parametros_totais = []

for _ in range(NUM_TENTATIVAS_OTIMIZACAO):
    study.optimize(objective, n_trials=1)
    clear_output()
    study.trials_dataframe().to_excel('triagem.xlsx')

<p style="text-align: justify;"> A partir das tentativas de modelos, o considerado melhor modelo, o que possui um menor valor de <i>RMSE</i>, teve seus hiperparâmetros exibidos salvo no <code>melhor_trial</code>. No caso abaixo, da forma que está salvo no github, perceba que o melhor modelo possui três camadas ocultas com pesos variados, das quais apenas a primeira camada não possui viés e

In [10]:
melhor_trial = study.best_trial

print(f"Número do melhor trial: {melhor_trial.number}")
print(f"Parâmetros do melhor trial: {melhor_trial.params}")
print(f"Valor do RMSE do melhor trial: {melhor_trial.value} MPa")

Número do melhor trial: 244
Parâmetros do melhor trial: {'n_layers': 3, 'c0': 7, 'bias0': False, 'c1': 11, 'bias1': True, 'c2': 12, 'bias2': True, 'bia out': False}
Valor do RMSE do melhor trial: 378.64423586770613 MPa


<p style="text-align: justify;"> A partir dos hiperparâmetros, criou-se um modelo de rede neural com esses hiperparâmetros, sendo dessa vez testada com o que foi chamado de <code>test_premium</code>, um conjunto de 10% dos exemplos dataset que não foi exposto ao modelo durante o treinamento e a otimização de hiperparâmetro. Testou-se nesse conjunto para averiguar o valor da métrica <i>RMSE</i> que seja mais confiável.

In [ ]:
# O número de dados de entrada (número de atributos) e o número de dados de saída são sempre os mesmos
num_dados_de_entrada = 5 
num_dados_de_saida = 1

# abaixo, foi salvo 
layers_number = int((len(melhor_trial.params) - 1) / 2) 

neuronios_camadas = []
vieses = []

for i in range(layers_number):
    neuronios_camadas.append(melhor_trial.params[f'c{i}'])
    vieses.append(melhor_trial.params[f'bias{i}'])
vieses.append(melhor_trial.params[f'bia out'])

# Criação do modelo com os hiperparâmetros utilizados
minha_mlp = MLP(
    num_dados_de_entrada, list(neuronios_camadas), list(vieses), num_dados_de_saida
)

dm = DataModule(TAMANHO_TESTE, SEMENTE_ALEATORIA, 1)

logger = TensorBoardLogger(save_dir=os.getcwd(), version=1, name="lightning_logs/")
treinador = L.Trainer(logger=False, enable_checkpointing=True, max_epochs=NUM_EPOCAS)

treinador.fit(minha_mlp, dm)dm.setup("test")

with torch.no_grad():
    X_true = dm.X_teste_premium

    y_true = dm.y_teste_premium
    y_true = dm.y_scaler.inverse_transform(y_true)

    y_pred = minha_mlp(X_true)
    y_pred = dm.y_scaler.inverse_transform(y_pred)

    RMSE = root_mean_squared_error(y_true, y_pred)

    print(RMSE)

197.1079938383723


<p style="text-align: justify;"> Acima, note que o modelo obteve um RMSE de 182.528 MPa, ainda menor que o resgistrado durante a otimização, o que é no mínimo interessante. Portanto, a partir da otimização de hiperparâmetros com 1000 tentativa variando o número de camadas ocultas e de neurônios por camada oculta, além da presença ou não de viés de neurônios a cada camada, foi possível obter um resultado ainda melhor, com um menor valor de RMSE.

### Referências:

[1] Cassar, D. R. 11 - Otimização de hiperparâmetros. Notebook Jupyter